In [1]:
import requests
import pandas as pd

In [31]:
# Define global variables for use in API call
with open('fred_keyfile.txt') as infile:
    fred_api_key = infile.read().strip()

base_url = "https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json"

sales_series_ids = ["DAUTOSAAR", "DLTRUCKSSAAR", "FAUTOSAAR", "FLTRUCKSSAAR", "HTRUCKSSAAR"]
gas_series_id = "GASREGCOVW"
gas_url = base_url + "&frequency=m"

In [20]:
# Get sales data for automobiles and trucks, merge them all into single dataframe
sales_df = pd.DataFrame({'date': []})

for series_id in sales_series_ids:
    res_json = requests.get(base_url.format(series_id=series_id, api_key=fred_api_key)).json()
    individual_df = pd.DataFrame(res_json['observations'], columns = ['date', 'value'])
    individual_df.rename(columns={'value': series_id}, inplace=True)
    
    sales_df = sales_df.merge(individual_df, how='outer', left_on='date', right_on='date')

In [39]:
sales_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 668 entries, 0 to 667
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          668 non-null    object
 1   DAUTOSAAR     668 non-null    object
 2   DLTRUCKSSAAR  668 non-null    object
 3   FAUTOSAAR     668 non-null    object
 4   FLTRUCKSSAAR  560 non-null    object
 5   HTRUCKSSAAR   668 non-null    object
dtypes: object(6)
memory usage: 36.5+ KB


In [30]:
sales_df.head(110)

,date,DAUTOSAAR,DLTRUCKSSAAR,FAUTOSAAR,FLTRUCKSSAAR,HTRUCKSSAAR
0,1967-01-01,7.837,1.235,0.629,NaN,0.359
1,1967-02-01,6.850,1.138,0.690,NaN,0.304
2,1967-03-01,7.220,1.237,0.740,NaN,0.359
3,1967-04-01,8.164,1.204,0.746,NaN,0.343
4,1967-05-01,7.961,1.184,0.782,NaN,0.344
...,...,...,...,...,...,...
105,1975-10-01,7.620,2.345,1.468,NaN,0.316
106,1975-11-01,7.895,2.508,1.370,NaN,0.288
107,1975-12-01,8.128,2.419,1.442,NaN,0.252
108,1976-01-01,8.453,2.589,1.288,0.182,0.302


In [36]:
res_json = requests.get(gas_url.format(series_id=gas_series_id, api_key=fred_api_key)).json()
gas_df = pd.DataFrame(res_json['observations'], columns=['date', 'value'])

In [38]:
# First data point is missing, keep all others
gas_df = gas_df.iloc[1:]
gas_df.head()

,date,value
1,1990-08-01,1.218
2,1990-09-01,1.258
3,1990-10-01,1.335
4,1990-11-01,1.324
5,1990-12-01,1.341


There's missing data in the Foreign Light Truck Sales column ('FLTRUCKSSAAR'). Replace this with zero.

In [43]:
sales_df = sales_df.fillna(0)

In [45]:
# Save pandas dataframes to csv files
sales_df.to_csv('vehicle_sales.csv', index=False)
gas_df.to_csv('gas_price.csv', index=False)